In [41]:
import pandas as pd
# from src.data_loader import YahooFinance as yfi
import yfinance as yf


In [70]:
symbol = "AAPL"

company = yf.Ticker(symbol)
bs = company.balancesheet
fin = company.financials


In [71]:
bsT = bs.transpose()
finT = fin.transpose()

data = bsT.join(finT)
data.columns

Index(['Treasury Shares Number', 'Ordinary Shares Number', 'Share Issued',
       'Net Debt', 'Total Debt', 'Tangible Book Value', 'Invested Capital',
       'Working Capital', 'Net Tangible Assets', 'Capital Lease Obligations',
       ...
       'Interest Expense Non Operating', 'Interest Income Non Operating',
       'Operating Income', 'Operating Expense', 'Research And Development',
       'Selling General And Administration', 'Gross Profit', 'Cost Of Revenue',
       'Total Revenue', 'Operating Revenue'],
      dtype='object', length=107)

In [72]:
ratios = pd.DataFrame(index=data.index)

# Profitability Ratios
ratios['Gross Profit Margin'] = data['Gross Profit'] / data['Total Revenue']
ratios['Operation Profit Margin'] = data['Operating Income'] / data['Total Revenue']
ratios['Net Profit Margin'] = data["Net Income"] / data['Total Revenue']
ratios['Return on Equity'] = data['Net Income'] / data['Total Revenue']
ratios['Return on Assets'] = data['Net Income'] / data['Stockholders Equity']

# Efficiency Ratios
ratios['Inventory Turnover Ratio'] = data["Cost Of Revenue"] / data['Inventory']
ratios['Receivables Turnover Ratio'] = data['Total Revenue'] / data['Receivables']

# Liability Ratio
ratios['Current Ratio'] = data['Current Assets'] / data['Current Liabilities']
ratios['Quick Ratio'] = data['Cash Cash Equivalents And Short Term Investments'] / data['Current Liabilities']
ratios['Cash Ratio'] = data['Cash And Cash Equivalents'] / data["Current Liabilities"]
ratios['Net Working Capital'] = data['Current Assets'] - data['Current Liabilities']
ratios['Operating Cash Flow Ratio'] = data['Operating Income'] / data['Current Liabilities']


In [67]:
ratios = ratios.transpose()

In [68]:

ratios.columns = ratios.columns.year
ratios.transpose().to_json()

'{"Gross Profit Margin":{"2024":0.4620634982,"2023":0.4413112958,"2022":0.4330963056,"2021":0.4177935963,"2020":null},"Operation Profit Margin":{"2024":0.3151022287,"2023":0.2982141227,"2022":0.302887444,"2021":0.2978237753,"2020":null},"Net Profit Margin":{"2024":0.2397125577,"2023":0.2530623426,"2022":0.2530964071,"2021":0.2588179336,"2020":null},"Return on Equity":{"2024":0.2397125577,"2023":0.2530623426,"2022":0.2530964071,"2021":0.2588179336,"2020":null},"Return on Assets":{"2024":1.6459350307,"2023":1.5607601455,"2022":1.9695887275,"2021":1.5007132668,"2020":null},"Inventory Turnover Ratio":{"2024":28.8707109525,"2023":33.8235665772,"2022":45.1973311767,"2021":32.3679331307,"2020":null},"Receivables Turnover Ratio":{"2024":5.9030388116,"2023":6.2849061245,"2022":6.4716076938,"2021":7.1024152526,"2020":null},"Current Ratio":{"2024":0.8673125765,"2023":0.9880116718,"2022":0.8793560286,"2021":1.0745531196,"2020":null},"Quick Ratio":{"2024":0.3694668692,"2023":0.4236174196,"2022":0.3

## Test Yfinance

In [90]:
from src.data_loader import data_source
from pandas import DataFrame

class YahooFinance(data_source):
    """Using Yahoo Finance as Source of Data"""

    def fetch(self, source) -> DataFrame:
        ticker_symbol: str = source
        company = yf.Ticker(ticker_symbol)
        bs = company.balance_sheet
        fs = company.financials
        data = bs.transpose().join(fs.transpose())
        if len(data):
            return data.transpose()

        logging.warning("Unable to fetch BS for %s" % ticker_symbol)

        if "." not in ticker_symbol:
            # Handling the Indian Ticker Symbols for yfinance
            for suffix in [".BO", ".NS"]:
                data = self.fetch(ticker_symbol + suffix)
                if len(data):
                    return data
        return DataFrame([])

    def get(self, source) -> DataFrame:
        df = self.fetch(source)

        if len(df)<1:
            logging.error("Unable to fetch data.")
        else:
            df = df / 100_00_00_000
            try:
                df.columns = df.columns.year
            except ValueError:
                logging.warning("The Periods are not defined properly")                
        return df

    def ratios(self, source):
        data = self.get(source).transpose()

        ratios = DataFrame(index=data.index)

        # Profitability Ratios
        ratios['Gross Profit Margin'] = data['Gross Profit'] / data['Total Revenue']
        ratios['Operation Profit Margin'] = data['Operating Income'] / data['Total Revenue']
        ratios['Net Profit Margin'] = data["Net Income"] / data['Total Revenue']
        ratios['Return on Equity'] = data['Net Income'] / data['Total Revenue']
        ratios['Return on Assets'] = data['Net Income'] / data['Stockholders Equity']

        # Efficiency Ratios
        ratios['Inventory Turnover Ratio'] = data["Cost Of Revenue"] / data['Inventory']
        ratios['Receivables Turnover Ratio'] = data['Total Revenue'] / data['Receivables']

        # Liability Ratio
        ratios['Current Ratio'] = data['Current Assets'] / data['Current Liabilities']
        ratios['Quick Ratio'] = data['Cash Cash Equivalents And Short Term Investments'] / data['Current Liabilities']
        ratios['Cash Ratio'] = data['Cash And Cash Equivalents'] / data["Current Liabilities"]
        ratios['Net Working Capital'] = data['Current Assets'] - data['Current Liabilities']
        ratios['Operating Cash Flow Ratio'] = data['Operating Income'] / data['Current Liabilities']

        return ratios.transpose()


temp = YahooFinance()
temp.ratios("AAPL")

,2024,2023,2022,2021,2020
Gross Profit Margin,0.462063,0.441311,0.433096,0.417794,NaN
Operation Profit Margin,0.315102,0.298214,0.302887,0.297824,NaN
Net Profit Margin,0.239713,0.253062,0.253096,0.258818,NaN
Return on Equity,0.239713,0.253062,0.253096,0.258818,NaN
Return on Assets,1.645935,1.560760,1.969589,1.500713,NaN
Inventory Turnover Ratio,28.870711,33.823567,45.197331,32.367933,NaN
Receivables Turnover Ratio,5.903039,6.284906,6.471608,7.102415,NaN
Current Ratio,0.867313,0.988012,0.879356,1.074553,NaN
Quick Ratio,0.369467,0.423617,0.313699,0.499191,NaN
Cash Ratio,0.169753,0.206217,0.153563,0.278449,NaN


## AlphaVantage

In [2]:
from requests import get
import os
from dotenv import load_dotenv
load_dotenv()

url = 'https://www.alphavantage.co/query'
param = {
    "function":"TOP_GAINERS_LOSERS",
    "apikey": os.getenv("ALPHAVANTAGE_API_KEY")
}

resp = get(url,param)
# resp.json()


In [6]:
param["function"] = "OVERVIEW"
param["symbol"] = "META"
resp = get(url,param)
resp.json()

{'Symbol': 'META',
 'AssetType': 'Common Stock',
 'Name': 'Meta Platforms Inc.',
 'Description': 'Meta Platforms, Inc. develops products that enable people to connect and share with friends and family through mobile devices, PCs, virtual reality headsets, wearables and home devices around the world. The company is headquartered in Menlo Park, California.',
 'CIK': '1326801',
 'Exchange': 'NASDAQ',
 'Currency': 'USD',
 'Country': 'USA',
 'Sector': 'TECHNOLOGY',
 'Industry': 'SERVICES-COMPUTER PROGRAMMING, DATA PROCESSING, ETC.',
 'Address': '1601 WILLOW ROAD, MENLO PARK, CA, US',
 'OfficialSite': 'https://investor.atmeta.com',
 'FiscalYearEnd': 'December',
 'LatestQuarter': '2025-03-31',
 'MarketCapitalization': '1794175664000',
 'EBITDA': '87979999000',
 'PERatio': '27.9',
 'PEGRatio': '2.357',
 'BookValue': '73.34',
 'DividendPerShare': '2.025',
 'DividendYield': '0.0029',
 'EPS': '25.58',
 'RevenuePerShareTTM': '67.35',
 'ProfitMargin': '0.391',
 'OperatingMarginTTM': '0.415',
 'Retu